In [240]:
import numpy as np
import pyaudio
import time
import math
from random import randrange
import librosa as rosa
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Convolution2D, Flatten, TimeDistributed

In [236]:

num_tones = 6

f = 440
srate = 11025
frame_count = 44100
clips = 2000
output = np.zeros((clips, frame_count))
Y = np.zeros((clips, num_tones))

for c in range(0, clips):
    #amplitudes = np.random.uniform(0, 1/num_tones, num_tones)
    amplitudes = np.repeat([1/num_tones], num_tones)
    ratios = np.random.uniform(1, randrange(2, 12), num_tones)
    ratios[0] = 1
    #rates = np.random.uniform(0.9996, 0.99999, num_tones)
    rates = np.repeat([0.9999], num_tones)
    #amplitudes = -np.sort(-amplitudes)
    t = 0
    #Y[c] = np.concatenate((amplitudes, ratios, rates))
    Y[c] = ratios
    
    for i in range(0, frame_count):
        for ot in range(0, num_tones):
            output[c, i] += amplitudes[ot]*math.sin(math.pi * t * f * ratios[ot] / srate)
        amplitudes = amplitudes * rates
        t = t + 1
#output = output * 1000
#output = output.astype('int16')



In [270]:
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=srate,
                output=True)

#stream.start_stream()
stream.write((output[5]*1000).astype('int16'))
print(Y[1])
stream.stop_stream()
stream.close()
p.terminate()

[ 1.         10.39384446  3.98724159  3.79010153  1.11528312  8.13715916]


In [274]:
MFCC = np.zeros((clips, 512, 87))
for i in range(clips):
    MFCC[i] = rosa.feature.melspectrogram(output[i], sr=11025, n_mels=512)
    #MFCC[i] = rosa.feature.mfcc(output[i], sr=11025, n_mfcc=20)

In [276]:
"""CNN network"""
MFCC_EXP = np.expand_dims(MFCC, axis=3)
cnn_model = Sequential()
cnn_model.add(Convolution2D(32, (4, 4), strides=(1, 1)))
cnn_model.add(Convolution2D(32, (2, 2), strides=(1, 1)))
cnn_model.add(Flatten())
cnn_model.add(Dense(units=36, activation='sigmoid'))
cnn_model.add(Dense(units=6))
cnn_model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])
cnn_model.fit(MFCC_EXP, Y, epochs=5)

Epoch 1/5
2000/2000 [==============================] - 73s 36ms/sample - loss: 8.4721 - accuracy: 0.1990
Epoch 2/5
2000/2000 [==============================] - 70s 35ms/sample - loss: 6.1781 - accuracy: 0.2025
Epoch 3/5
2000/2000 [==============================] - 65s 32ms/sample - loss: 5.2233 - accuracy: 0.2035
Epoch 4/5
2000/2000 [==============================] - 76s 38ms/sample - loss: 4.7720 - accuracy: 0.2050
Epoch 5/5
2000/2000 [==============================] - 67s 33ms/sample - loss: 4.5602 - accuracy: 0.2045


In [249]:
"""LSTM network"""

lstm_model = Sequential()
lstm_model.add(LSTM(6))
lstm_model.add(Dense(units=256, activation='sigmoid'))
lstm_model.add(Dense(units=6))
lstm_model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])
lstm_model.fit(MFCC, Y, epochs=5)

Epoch 1/5
2000/2000 [==============================] - 8s 4ms/sample - loss: 9.7661 - accuracy: 0.2060
Epoch 2/5
2000/2000 [==============================] - 6s 3ms/sample - loss: 4.6973 - accuracy: 0.2000
Epoch 3/5
2000/2000 [==============================] - 6s 3ms/sample - loss: 4.3957 - accuracy: 0.1975
Epoch 4/5
2000/2000 [==============================] - 6s 3ms/sample - loss: 4.3763 - accuracy: 0.2025
Epoch 5/5
2000/2000 [==============================] - 6s 3ms/sample - loss: 4.3750 - accuracy: 0.1955


In [277]:
cnn_o = cnn_model.predict(MFCC_EXP)
lstm_o = lstm_model.predict(MFCC)
print(cnn_o[41])
print(lstm_o[41])
print(Y[41])

[0.90120107 3.2703967  3.2859488  3.3186808  3.4688032  3.5348306 ]
[0.99735785 3.6584399  3.7307768  3.757041   3.762695   3.8094149 ]
[1.         2.4564495  1.74668936 1.47902497 2.16641377 2.23719866]


In [281]:
g = 10
print(lstm_o[g])
print(Y[g])

[0.99735785 3.6584399  3.7307768  3.757041   3.762695   3.8094149 ]
[1.         7.57747292 6.17471285 3.72331313 5.94116858 9.82988944]


In [282]:
cnn_diff_sum = 0
lstm_diff_sum = 0
for i in range(100):
    for j in range(6):
        cnn_diff_sum += cnn_o[i, j] - Y[i, j]
        lstm_diff_sum += lstm_o[i, j] - Y[i, j]

print(cnn_diff_sum / 600, "...", lstm_diff_sum / 600)

-0.5029616348644725 ... -0.18209069139765036


In [271]:
recreation = np.zeros((frame_count))
t = 0
ratios = cnn_o[5]
ratios[0] = 1
amplitudes = np.repeat([1/num_tones], num_tones)
rates = np.repeat([0.9999], num_tones)
for i in range(0, frame_count):
    for ot in range(0, num_tones):
        recreation[i] += amplitudes[ot]*math.sin(math.pi * t * f * ratios[ot] / srate)
    amplitudes = amplitudes * rates
    t = t + 1
    
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=srate,
                output=True)

stream.write((recreation*1000).astype('int16'))
stream.stop_stream()
stream.close()
p.terminate()